<a href="https://colab.research.google.com/github/Halogen/Datarobot_Example/blob/main/Datarobot_Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Importing neessary files from drive for the mlops developer package
from google.colab import drive
drive.mount('/content/drive')
%cp - av /content/drive/MyDrive/datarobot_mlops_package-7.2.1-1074 (4).tar.gz

Mounted at /content/drive
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cp - av /content/drive/MyDrive/datarobot_mlops_package-7.2.1-1074 (4).tar.gz'


In [ ]:
%cp -av '/content/drive/MyDrive/datarobot_mlops_package-7.2.1-1074 (4).tar.gz' /content/

'/content/drive/MyDrive/datarobot_mlops_package-7.2.1-1074 (4).tar.gz' -> '/content/datarobot_mlops_package-7.2.1-1074 (4).tar.gz'


In [ ]:
#Un-tarring files
import tarfile
!tar xvzf  '/content/datarobot_mlops_package-7.2.1-1074 (4).tar.gz'

In [ ]:
#Adding
import os
!mkdir -p tmp/ta
!ls -l
%cd '/content/datarobot_mlops_package-7.2.1/'
!pwd

total 183080
drwxr-xr-x 9 1001 1001      4096 Jun 11 02:05  datarobot_mlops_package-7.2.1
-rw------- 1 root root 187452328 Jul 28 04:58 'datarobot_mlops_package-7.2.1-1074 (4).tar.gz'
drwx------ 6 root root      4096 Jul 30 00:24  drive
drwxr-xr-x 1 root root      4096 Jul 16 13:20  sample_data
drwxr-xr-x 3 root root      4096 Jul 30 00:26  tmp
/content/datarobot_mlops_package-7.2.1
/content/datarobot_mlops_package-7.2.1


In [ ]:
#InstallinG MLOps library ( Making sure to install the one for connected client as well!)
!pip install lib/datarobot_mlops-7.2.1-py2.py3-none-any.whl
!pip install lib/datarobot_mlops_connected_client-7.2.1-py2.py3-none-any.whl

In [ ]:
#Editing config yaml file with rabbit mq spooler , API and URL
%cp -av '/content/datarobot_mlops_package-7.2.1/conf/mlops.agent.conf.yaml' conf/


cp: '/content/datarobot_mlops_package-7.2.1/conf/mlops.agent.conf.yaml' and 'conf/mlops.agent.conf.yaml' are the same file


In [ ]:
#start agent
!./bin/start-agent.sh

INFO: AGENT_CONFIG_YAML=/content/datarobot_mlops_package-7.2.1/conf/mlops.agent.conf.yaml
INFO: AGENT_LOG_PROPERTIES=/content/datarobot_mlops_package-7.2.1/conf/mlops.log4j2.properties
INFO: AGENT_JVM_OPT=-Xmx1G
INFO: AGENT_JAR_PATH=/content/datarobot_mlops_package-7.2.1/lib/mlops-agent-7.2.1.jar
INFO: AGENT_LOG_PATH=/content/datarobot_mlops_package-7.2.1/logs/mlops.agent.log

Starting MLOps-Agent


DataRobot MLOps-Agent is running.


In [ ]:
cat /content/datarobot_mlops_package-7.2.1/logs/mlops.agent.log


In [ ]:
from datarobot.mlops.mlops import MLOps
from datarobot.mlops.common.enums import OutputType
from datarobot.mlops.common.exception import DRConnectedException
from datarobot.mlops.constants import Constants
from datarobot.mlops.connected.client import MLOpsClient

# Uploading Pickle file ( A pre trained model )
###### We need deployment ID  and Model_ID from datarobot and google project and pubsub topic id

In [ ]:
import time
import random
import argparse
import csv
import pytz
import datetime
import pandas as pd

deployment_id = '61013bd857c7dc0a7d94c699'
model_id = '60f27ec00675a525ba820a32'
project_id = "DatarobotInterview"
topic_id = "Datarobot123"
#spool_dir = "/content/datarobot_mlops_package-7.2.1/tmp/ta"
MLOPS_FILESYSTEM_MAX_FILE_SIZE = 104857600
MLOPS_FILESYSTEM_MAX_NUM_FILES = 5

# name of the file that contains actuals
ACTUALS_OUTPUT_FILE = "/content/datarobot_mlops_package-7.2.1/Model/Diabetes_test - diabetes.csv"

!export MLOPS_FILESYSTEM_MAX_FILE_SIZE
!export MLOPS_FILESYSTEM_MAX_NUM_FILES

In [ ]:
# MLOPS: initialize the MLOps instance
mlops = MLOps() \
        .set_deployment_id(deployment_id) \
        .set_model_id(model_id) \
        .set_pubsub_spooler(project_id, topic_id) \
        .init()

In [ ]:
# load pickle file with model pipeline
model = pd.read_pickle ('/content/datarobot_mlops_package-7.2.1/Model/model.pkl')
features_df_scoring = pd.read_csv('/content/datarobot_mlops_package-7.2.1/Model/Diabetes_test - diabetes.csv')

# Get predictions
start_time = time.time()
predictions = model.predict(features_df_scoring).tolist()
end_time = time.time()

# number of predictions
num_predictions = len(predictions)

# time required to generate the predictions
prediction_time = end_time - start_time

In [ ]:
# Generate assocation ids for the predictions so we can match them with actuals
# this is necessary for accuracy monitoring
# The association ids are generated by taking the current time and appending a row counter to it
def generate_unique_association_ids(num_samples):
    ts = time.time()
    return ["x_{}_{}".format(ts, i) for i in range(num_samples)]

association_ids = generate_unique_association_ids(len(features_df_scoring))


In [ ]:
# MLOPS: report the number of predictions in the request and the execution time
MLOps.report_deployment_stats(num_predictions=num_predictions, execution_time_ms=prediction_time)

In [ ]:
MLOps.report_predictions_data(features_df=features_df_scoring, predictions=predictions)

In [ ]:
num_predictions

442

In [ ]:
prediction_time

0.004504680633544922